In [1]:
import sys
sys.path.append('..')

# ChEMBL

**ChEMBL** jest bazą danych aktywności związków chemicznych. Tutaj trafiają dane z eksperymentów biologicznych, które później można wykorzystać do wytrenowania modeli uczenia maszynowego. Trzeba jednak zwrócić uwagę, że baza ta jest ręcznie poprawiana, a dane gromadzone są z różnych publikacji. Co za tym idzie, jakość danych nie jest idealna, a czasem nawet ten sam związek może mieć drastycznie różne pomiary aktywności. Zdarzają się dane błędne, a metody pomiaru aktywności mogą się różnić między zespołami.

Żeby wartości związane z aktywnością pojawiły się w bazie danych, zespół chemików lub biologów musi przeprowadzić **test biologiczny** (ang. biological assay, bioassay). W trakcie takiego testu związek chemiczny może na przykład być podany bezpośrednio do białka, z którym ma oddziaływać, lub też na komórki. Następnie odpowiednimi narzędziami mierzona jest aktywność związku. W bazie ChEMBL znajdziemy między innymi wartości aktywności podane jako:

- **EC$_{50}$/IC$_{50}$** - minimalne stężenie, przy którym zachodzi połowa efektu lub inhibicji (blokady działania); w tym przypadku mniejsza liczba oznacza większą aktywność,
- **$\text{K}_\text{i}$** - stała inhibicji, która wyznacza siłę wiązania się z białkiem; również mniejsze wartości to większa aktywność.

Często wartości aktywności konwertowane są przy pomocy logarytmu i oznaczane dodatkową literą p, np. **pIC$_{50}$** to ujemny logarytm dziesiętny z IC$_{50}$. Pozwala to zmniejszyć wpływ dużych wartości. W tym wypadku większe wartości oznaczają większą aktywność.

![assay](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a0/96_well_plate.jpg/1599px-96_well_plate.jpg)

Przykładowe cele biologiczne (do wyboru w zadaniach poniżej):

- MAO-B (monoaminooksydaza B) - odpowiedzialna między innymi za chorobę Parkinsona
- ACE2 (enzym konwertujący angiotensynę typu 2) - odpowiedzialny za wiązanie się wirusa SARS-CoV-2
- COX-2 (cyklooksygenaza 2) - cel niesteroidowych leków przeciwzapalnych
- 5-HT3 (receptor serotoninowy 3) - częsty cel leków przeciwwymiotnych
- hERG - kanał potasowy w sercu odpowiedzialny za toksyczność

*Na marginesie: Użycie danych hERG będzie dobrym wstępem do projektu nr 7: Przewidywanie kardiotoksyczności związków*

W pierwszym zadaniu będziemy starać się użyć dostępnych danych o aktywnościach związków wobec wybranego celu biologicznego, aby stworzyć model przewidujący tę aktywność. Jeśli swoje przewidywania opieramy na strukturze związku (mamy dostęp do SMILES, który opisuje właśnie strukturę), to swój model możemy nazwać modelem **QSAR** (Quantitative Structure-Activity Relationship).

**Zadanie:** Znajdź związki z danymi aktywności do wybranego celu biologicznego w bazie ChEMBL. Stwórz z nich zbiór danych i wytrenuj model przewidujący aktywność biologiczną. Możesz użyć kodu modelu z poprzednich zajęć.

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem

In [78]:
pd.set_option('display.max_columns', None)
data_path = "..."  # TODO: You should change this
df = pd.read_csv(data_path, sep=';')
df.head()

In [77]:
activity_type = 'IC50'  # TODO: You can change this
relevant_columns = ['Molecule ChEMBL ID', 'Molecule Name', 'Molecular Weight', '#RO5 Violations', 'Smiles',
                    'Standard Value', 'Standard Units', 'pChEMBL Value']
df = df[(df['Standard Type'] == activity_type) & (df['Standard Relation'].str.contains('='))][relevant_columns]
df

In [76]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2)
len(df_train), len(df_test)

In [72]:
# TODO: Use your training script

class Featurizer:
    def __init__(self, y_column, **kwargs):
        self.y_column = y_column
        self.__dict__.update(kwargs)
    
    def __call__(self, df):
        raise NotImplementedError()
        

class ECFPFeaturizer(Featurizer):
    def __init__(self, y_column, radius=2, length=1024, **kwargs):
        self.radius = radius
        self.length = length
        super().__init__(y_column, **kwargs)
    
    def __call__(self, df):
        fingerprints = []
        labels = []
        for i, row in df.iterrows():
            y = row[self.y_column]
            smiles = row.Smiles
            mol = Chem.MolFromSmiles(smiles)
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, self.radius, nBits=self.length)
            fingerprints.append(fp)
            labels.append(y)
        fingerprints = np.array(fingerprints)
        labels = np.array(labels)
        return fingerprints, labels


from sklearn.ensemble import RandomForestRegressor

featurizer = ECFPFeaturizer(y_column='Standard Value')
X_train, y_train = featurizer(df_train)
X_test, y_test = featurizer(df_test)

model = RandomForestRegressor()
model.fit(X_train, y_train)
model.score(X_test, y_test)

Klasyczny QSAR wykonywany był na związkach o podobnych strukturach, aby skuteczność metody była jak największa. Badano na przykład wpływ różnych podstawników na zmianę aktywności związku. Obecnie w bazach takich jak ChEMBL znaleźć można wiele serii związków działających na dany cel, tj. wiele podobnych struktur opracowanych na drodze do znalezienia aktywnego związku. Losowy podział danych nie zawsze jest więc optymalny, bo do zbioru testowego mogą trafić analogi związków treningowych (bardzo podobne związki). Często zamiast tego używa się **podziału względem czasu** (bardziej złożone modyfikacje były wprowadzane później) oraz **względem scaffoldu**/rdzenia związku (upewniamy się, że żaden scaffold nie znajduje się jednocześnie w zbiorze treningowym i testowym).

![scaffolds](https://media.springernature.com/full/springer-static/image/art%3A10.1007%2Fs00894-019-3950-6/MediaObjects/894_2019_3950_Fig1_HTML.png)

**Zadanie:** Jak wyniki przewidywania zmieniają się, gdy zostanie zmieniona metoda podziału danych?

In [ ]:
# TODO: scaffold or time split

Wcześniej wspomniałem już, że logarytm często w chemii może pomóc w radzeniu sobie z dużymi wartościami liczbowymi. Okazuje się, że równie pomocna może być zamiana wartości ciągłych na 2 klasy: związki aktywne i nieaktywne. Wybiera się wówczas próg na wartościach, który rozdziela te dwie klasy, a problem zamienia się w problem klasyfikacji. Pozbywamy się w ten sposób między innymi możliwych błędów pomiaru, które mogą wynosić nawet kilka rzędów wielkości.

**Zadanie:** Czy użycie przekształconych (logarytm) albo zbinaryzowanych wartości aktywności poprawia predykcje?

*Na marginesie: To zadanie jest wstępem do projektu nr 2: Regresja vs. Klasyfikacja.*

In [75]:
activity_threshold = 100
df['class'] = (df['Standard Value'] <= activity_threshold).astype(int)
df

In [ ]:
# TODO: classification setup

**Zadanie:** Czy znajdujesz jakieś nieścisłości w danych? Na przykład znajdujesz wartości odstające albo SMILES zawiera więcej niż jeden związek? Jak zaradzić tym problemom?

In [74]:
# Hint: A dot in the SMILES string indicates multiple compounds (this is a separator)
df[df.Smiles.str.contains('\.')]

# TODO: Perform Exploratory Data Analysis

# ZINC

**ZINC** jest bazą komercyjnie dostępnych związków chemicznych. Dane są przeznaczone przede wszystkim do tzw. **wirtualnego skriningu**, czyli badania aktywności związków modelami *in silico* (zaimplementowanymi na komputerze). Związki ocenione najlepiej przez modele mogą zostać następnie zakupione i przetestowane *in vitro* (na komórkach lub biomolekułach) lub *in vivo* (na żywych organizmach).

**Zadanie:** Pobierz mały podzbiór związków dostępnych do kupienia i przeprowadź skrining wirtualny przy użyciu wytrenowanego powyżej modelu. Które związki powinny zostać wybrane do zsyntetyzowania?

In [ ]:
# TODO: inference: (1) load data (2) convert representation (3) predict activity (4) show drug candidates

**Zadanie:** Jak bardzo podobne są aktywne związki do tych w zbiorze treningowym? Czy podobieństwo związków wpłynąć powinno na decyzję odnośnie wyboru związków do syntezy?

In [ ]:
# TODO: Check structural similarity

**Zadanie:** Usuń mało obiecujące propozycje modelu przez odfiltrowanie "podejrzanych" struktur (SMARTS).

In [73]:
df_alerts = pd.read_csv('../data/alerts/SureChEMBL.csv')
df_alerts

In [ ]:
# TODO: Use SMARTS to filter out compounds

# PubChem

**PubChem** jest otwartą bazą danych związków chemicznych, zawierającą różne informacje o tych związkach, jak na przykład formy krystaliczne, czy różne własności ADMET (Absorption Distribution Metabolism Excretion Toxicity). Umożliwia też łatwe wyszukiwanie związków, w tym po SMARTS-ach i ręcznie narysowanych strukturach. 

**Zadanie:** Spróbuj znaleźć informacje o topowych związkach Twojego skriningu w bazie PubChem.